In [43]:
!pip install deims
!pip install xmltodict
!pip install json2xml
!pip install dict2xml
!pip install dicttoxml



[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 5.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [50]:
import uuid
import json
import xmltodict
import deims
import datetime
import os
from json2xml import json2xml
from json2xml.utils import readfromurl, readfromstring, readfromjson
import xml.etree.ElementTree as ET

import dicttoxml


In [51]:
with open('getcapabilities_1.3.0.xml') as fd:
    doc = xmltodict.parse(fd.read())

root = ET.Element('MD_Metadata')
for layer in doc['WMS_Capabilities']['Capability']['Layer']['Layer']:

    record_uuid = str(uuid.uuid4())

    keywords_to_be_printed = []
    authors = None
    # process keywords for site information and temporal extent
    for keyword in layer['KeywordList']['Keyword']:
        uri = None

        if "project: " in keyword:
            if "e-shape" in keyword:
                uri = "https://doi.org/10.3030/820852"
                keyword = "e-shape"

            if "AGAME" in keyword:
                keyword = "AGAME"

        if "variable: " in keyword:
            data_product_type = keyword[10:]
            if data_product_type == "Phenology":
                keyword = "Phenology"
                uri = "https://vocabs.lter-europe.net/EnvThes/21647"
                authors = [{
                  "individualName": "Saverio Vicario",
                  "organisationName": "CNR",
                  "role": "author",
                  "email": "saverio.vicario@iia.cnr.it",
                  "nameIdentifier": "https://orcid.org/0000-0003-1140-0483",
                }]
            if data_product_type == "Hydroperiod":
                keyword = "Hydroperiod"
                authors = [{
                  "individualName": "Ioannis Manakos",
                  "organisationName": "CERTH",
                  "role": "author",
                  "email": "imanakos@iti.gr",
                  "nameIdentifier": "https://orcid.org/0000-0001-6833-294X",
                }]
            if data_product_type == "GPP":
                keyword = "GPP"
                uri = "http://vocabs.lter-europe.net/EnvThes/21074"
                authors = [
                    {
                      "individualName": "Mario Alberto Fruentes-Monjarez",
                      "organisationName": "Deltares",
                      "role": "author",
                    },
                    {
                      "individualName": "Valeria Mobilia",
                      "organisationName": "Deltares",
                      "role": "author",
                    },
                ]
            if data_product_type == "snowcover":
                uri = "https://vocabs.lter-europe.net/EnvThes/21559"
                keyword = "snowCover"
                authors = [
                    {
                      "individualName": "Maria Adamo",
                      "organisationName": "CNR",
                      "role": "author",
                      "email": "adamo@iia.cnr.it",
                      "nameIdentifier": "https://orcid.org/0000-0003-3030-4884",
                    },
                    {
                      "individualName": "Chiara Richiardi",
                      "organisationName": "CNR",
                      "role": "author",
                      "email": "'chiara.richiardi@iia.cnr.it",
                      "nameIdentifier": "https://orcid.org/0000-0002-2370-7768",
                    }
                ]

        if "site: " in keyword:
            deims_suffix = keyword[6:]
            #print(site_record)
            uri = "https://deims.org/" + deims_suffix
            site_record = deims.getSiteById(deims_suffix)

            try:
              keyword = site_record["title"]
            except:
              print(layer['Title'])
              print(deims_suffix)

        if "https://doi.org/" in keyword:
            doi = {
                "name": "B2Share",
                "description": "Download data on B2Share",
                "url": keyword
            }

        if "time: " in keyword:
            time_string = keyword[6:]

            if "-" in time_string:
                time_values = time_string.split('-')
                period = {
                    "begin": time_values[0] + '-01-01',
                    "end": time_values[1] + '-01-01'
                }

            else:
                # might be necessary to have start/end date with days and months, not just year
                period = {"begin": time_string}

        if uri:
            formatted_keyword = {
                "value": keyword,
                "uri": uri
            }
        else:
            formatted_keyword = {
                "value": keyword
            }

        keywords_to_be_printed.append(formatted_keyword)

    #print(deims_suffix)

    # fetch EPSG code
    for value in layer['CRS']:
        if "EPSG:" in value:
            epsg_code = value[5:]

    # boundaries
    westBoundLongitude = layer['EX_GeographicBoundingBox']["westBoundLongitude"]
    eastBoundLongitude = layer['EX_GeographicBoundingBox']["eastBoundLongitude"]
    southBoundLatitude = layer['EX_GeographicBoundingBox']["southBoundLatitude"]
    northBoundLatitude = layer['EX_GeographicBoundingBox']["northBoundLatitude"]

    record = {
        "id": record_uuid,
        "uri": "https://catalogue.lter-europe.net/id/" + record_uuid,
        "type": "signpost",
        "title": layer['Title'],
        "description": layer['Abstract'], # we could add a sentence that the metadata record was generated automatically
        "metadataDate": str(datetime.datetime.now()),
        "resourceIdentifiers": {"code": "TBD/" + record_uuid},
        "descriptiveKeywords": [{"keywords": keywords_to_be_printed}],
        "responsibleParties": authors,
        "temporalExtents": period,
        "boundingBoxes": [
             {
              "westBoundLongitude": westBoundLongitude,
              "eastBoundLongitude": eastBoundLongitude,
              "southBoundLatitude": southBoundLatitude,
              "northBoundLatitude": northBoundLatitude,
              "bounds":  {
                    "type": "Feature",
                    "properties": {},
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [
                            [
                                [westBoundLongitude, southBoundLatitude], 
                                [westBoundLongitude, northBoundLatitude], 
                                [eastBoundLongitude, northBoundLatitude], 
                                [eastBoundLongitude, southBoundLatitude], 
                                [westBoundLongitude, southBoundLatitude]
                            ]
                        ]
                    }
                },
              "coordinates": f"[[[{westBoundLongitude}, {southBoundLatitude}], [{westBoundLongitude}, {northBoundLatitude}], [{eastBoundLongitude}, {northBoundLatitude}], [{eastBoundLongitude}, {southBoundLatitude}], [{westBoundLongitude}, {southBoundLatitude}]]]"
            }
        ],
        "supplemental": [doi],
        "service": {
            "type": "WMS"
        },
        "mapDataDefinition": {
            "data": [
                {
                    "path": "https://elter.datalabs.ceh.ac.uk/geoserver/ows?version=1.3.0",
                    "epsgCode": epsg_code,
                    "type": "POLYGON",
                    "features": {
                        "name": layer['Name'],
                        "label": layer['Title'],
                        "style": {
                            "colour": "#000000"
                        }
                    },
                    "bytetype": False
              }
            ]
        },
        "resourceType": {
            "value": "signpost"
        },
        "notGEMINI": False,
        "deimsSites": [{
            "title": site_record["title"],
            "id": deims_suffix,
            "url": "https://deims.org/" + deims_suffix
        }],
        "linkedDocument": False,
        "mapViewable": False,
        "incomingCitationCount": 0,
        "authors": authors
    }

    filename = record_uuid + ".xml"
    base_dir = os.getcwd() + '/files_for_csw'
    destination_path = os.path.join(base_dir, filename)
    
    # Convert a JSON to XML
    xml_object = json2xml.Json2xml(record).to_xml()
    xml = dicttoxml.dicttoxml(record, return_bytes=False, attr_type=False, include_encoding=False, custom_root='gmd:MD_Metadata')
    
    with open(destination_path, "w+") as f:
        f.write(xml)


KeyboardInterrupt: 